# Online Feature Store

This notebook includes examples of how to interact with the **Online** Feature Store in Hopsworks. The online feature store stores a subset of the feature data for real-time queries, suited for serving client-facing models. 

The online feature store contrasts to the **offline** feature store. The offline feature store contains historical data. The offline feature data is stored in Hive, a storage engine suited for large scale batch processing of data (such as *training* a machine learning model). On the other hand, the online feature store uses MySQL-Cluster database as the backend, a storage engine suited for smaller datasets that need to be queried in real-time.

### Imports

In [1]:
from hops import featurestore

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1569258228210_0011,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


### Get JDBC Connection to the Online Feature Store

If your project's feature store has the online feature store enabled, there will be a storage connector for each user to access the online feature store. The storage connector can be accessed using the utility method `get_online_featurestore_connector()` in the Python SDK. The storage connector includes information about the JDBC connection, the password, port, host, and username etc.

In [2]:
storage_connector = featurestore.get_online_featurestore_connector()

In [3]:
storage_connector.connection_string

'jdbc:mysql://10.0.2.15:3306/demo_featurestore_admin000'

In [4]:
storage_connector.name

'demo_featurestore_admin000_meb1_onlinefeaturestore'

In [5]:
storage_connector.description

'JDBC connection to Hopsworks Project Online Feature Store NDB Database for user: demo_featurestore_admin000_meb1'

In [6]:
storage_connector.arguments

'password=YTnQHFxNHwMlEpZboyJCgpZFSqyyKgQHXnUHJzSrVNhOslGRqKifTmzvRnhudipF,user=demo_featurestore_admin000_meb1'

### Create a Feature Group with Online Feature Serving Enabled.

When a feature group has online feature serving enabled, it means that its data will be stored in both Hive (for historical queries) and MySQL Cluster (for online queries). To enable online feature serving of a feature group simply set the flag `online=True` when creating a feature group, as illustrated below.

#### Create Sample Data

In [7]:
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, IntegerType, FloatType
sqlContext = SQLContext(sc)
schema = StructType([StructField("id", IntegerType(), True),
                     StructField("val_1", FloatType(), True),
                     StructField("val_2", IntegerType(), True)
                        ])
sample_df = sqlContext.createDataFrame([(999, 41251.52, 1), (998, 1319.4, 8), (997, 21219.1, 2)], schema)

In [8]:
sample_df.show(5)

+---+--------+-----+
| id|   val_1|val_2|
+---+--------+-----+
|999|41251.52|    1|
|998|  1319.4|    8|
|997| 21219.1|    2|
+---+--------+-----+

#### Save the Sample Data as a Feature group with online feature serving enabled


In [9]:
featurestore.create_featuregroup(sample_df, "online_featuregroup_test", online=True, primary_key="id")

computing descriptive statistics for : online_featuregroup_test, version: 1
computing feature correlation for: online_featuregroup_test, version: 1
computing feature histograms for: online_featuregroup_test, version: 1
computing cluster analysis for: online_featuregroup_test, version: 1
Registering feature metadata...
Registering feature metadata... [COMPLETE]
Writing feature data to offline feature group (Hive)...
Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Writing feature data to offline feature group (Hive)... [COMPLETE]
Writing feature data to online feature group (MySQL)...
Writing feature data to online feature group (MySQL)... [COMPLETE]
Feature group created successfully

When creating a feature group, the spark dataframe is used to infer the data-schema and the feature types. The data schema is then used to create a Hive table (for offline data) and a MySQL table (for online data). If you want to have more control over the feature types for the MySQL table (e.g length of a varchar column) you can pass in the types in the optional argument `online_types`, which takes a dict of the form `feature_name --> feature_type`.

In [10]:
sample_df_types_test = sample_df.withColumnRenamed(
    "val_1", "val_1_type_test").withColumnRenamed(
    "val_2", "val_2_type_test")

In [11]:
featurestore.create_featuregroup(sample_df_types_test, "online_featuregroup_test_types", 
                                 online=True, primary_key="id", online_types = {"val_1": "DECIMAL"})

computing descriptive statistics for : online_featuregroup_test_types, version: 1
computing feature correlation for: online_featuregroup_test_types, version: 1
computing feature histograms for: online_featuregroup_test_types, version: 1
computing cluster analysis for: online_featuregroup_test_types, version: 1
Registering feature metadata...
Registering feature metadata... [COMPLETE]
Writing feature data to offline feature group (Hive)...
Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Writing feature data to offline feature group (Hive)... [COMPLETE]
Writing feature data to online feature group (MySQL)...
Writing feature data to online feature group (MySQL)... [COMPLETE]
Feature group created successfully

### Read Features from Online Feature Store

The same methods for reading the offline feature store can be used to read from the online feature store by setting the argument `online=True`. **However, NOTE**: as the online feature store is supposed to be used for feature serving, it should be queried with primary-key lookups for getting the best performance. In fact, it is highly discouraged to use the online feature serving for doing full-table-scans. If you find yourself frequently needing to use `get_featuregroup(online=True)` to get the entire feature group (full-table scan), you are probably better of using the offline feature store. The online feature store is intended for quick primary key lookups, not data analysis.

To make the migration from the regular offline-featurestore API to the online-featurestore simple, for each example of reading from the online featurestore below, there is an accompanying example of reading from the offline feature store.

#### Free-Text SQL Query for the Online Feature Store

Featuregroups are stored as tables with the naming `featuregroupname_version` as Hive tables for offline features, and MySQL tables for online features.

In [12]:
#primary key lookup in MySQL
df = featurestore.sql("SELECT val_1 FROM online_featuregroup_test_1 WHERE id=999", online=True)

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Running sql: SELECT val_1 FROM online_featuregroup_test_1 WHERE id=999 against online feature store

In [13]:
df.show(5)

+-------+
|  val_1|
+-------+
|41251.5|
+-------+

#### Free-Text SQL Query for the Offline Feature Store

In [14]:
# primary key lookup in Hive
df = featurestore.sql("SELECT val_1 FROM online_featuregroup_test_1 WHERE id=999", online=False) 

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Running sql: SELECT val_1 FROM online_featuregroup_test_1 WHERE id=999 against offline feature store

In [15]:
df.show(5)

+--------+
|   val_1|
+--------+
|41251.52|
+--------+

#### Read Online Version of Feature Group

In [16]:
df = featurestore.get_featuregroup("online_featuregroup_test", online=True)

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM online_featuregroup_test_1 against online feature store

In [17]:
df.show(5)

+---+-------+-----+
| id|  val_1|val_2|
+---+-------+-----+
|997|21219.1|    2|
|999|41251.5|    1|
|998| 1319.4|    8|
+---+-------+-----+

#### Read Offline Version of Feature Group

In [18]:
df = featurestore.get_featuregroup("online_featuregroup_test")

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM online_featuregroup_test_1 against offline feature store

In [19]:
df.show(5)

+---+--------+-----+
| id|   val_1|val_2|
+---+--------+-----+
|998|  1319.4|    8|
|997| 21219.1|    2|
|999|41251.52|    1|
+---+--------+-----+

#### Read Online Version of individual Feature

In [20]:
df = featurestore.get_feature("val_1", online=True)

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Logical query plan for getting 1 feature from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT val_1 FROM online_featuregroup_test_1 against online feature store

In [21]:
df.show(5)

+-------+
|  val_1|
+-------+
|21219.1|
|41251.5|
| 1319.4|
+-------+

#### Read Offline Version of individual Feature

In [22]:
df = featurestore.get_feature("val_1")

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Logical query plan for getting 1 feature from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT val_1 FROM online_featuregroup_test_1 against offline feature store

In [23]:
df.show(5)

+--------+
|   val_1|
+--------+
|  1319.4|
| 21219.1|
|41251.52|
+--------+

#### Read Online Version of a list of individual Features

The featues can potentially span multiple feature groups, as long as all feature groups have online serving enabled, the feature store query planner will join the features on the fly.

In [24]:
df = featurestore.get_features(["val_1", "val_2"], online=True)

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Logical query plan for getting 2 features from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT val_1, val_2 FROM online_featuregroup_test_1 against online feature store

In [25]:
df.show(5)

+-------+-----+
|  val_1|val_2|
+-------+-----+
|21219.1|    2|
|41251.5|    1|
| 1319.4|    8|
+-------+-----+

#### Read Offline Version of a list of individual Features

In [26]:
df = featurestore.get_features(["val_1", "val_2"])

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Logical query plan for getting 2 features from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT val_1, val_2 FROM online_featuregroup_test_1 against offline feature store

In [27]:
df.show(5)

+--------+-----+
|   val_1|val_2|
+--------+-----+
|  1319.4|    8|
| 21219.1|    2|
|41251.52|    1|
+--------+-----+

#### List all feature groups that have online feature serving enabled

In [28]:
featurestore.get_featuregroups(online=True)

['online_featuregroup_test_types_1', 'online_featuregroup_test_1']

#### List all features that have online feature serving enabled

In [29]:
featurestore.get_features_list(online=True)

['id', 'val_1_type_test', 'val_2_type_test', 'id', 'val_1', 'val_2']

### Enable Online Feature Serving for a Feature Group that is Offline-Only

By default when a feature group is created with `create_featuregroup()`, the feature group will not have online serving enabled, all data will be stored in the offline feature group (Hive). To create a feature group with online serving, pass the flag `online=True` to `create_featuregroup()` (an example is provided in the beginning of this notebook).

If you want to enable online feature serving for a feature group dynamically, after the feature group have been created, you can use the API call `enable_featuregroup_online` (this will create a MySQL table in the backend). Conversely, if you want to disable online feature serving, use the API call `disable_featuregroup_online` (this will drop the MySQL table in the backend).

#### Create Feature Group without Online Feature Serving Enabled

In [30]:
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, IntegerType, FloatType
sqlContext = SQLContext(sc)
schema = StructType([StructField("id", IntegerType(), True),
                     StructField("test_col_1", FloatType(), True),
                     StructField("test_col_2", IntegerType(), True)
                        ])
sample_df = sqlContext.createDataFrame([(999, 41251.52, 1), (998, 1319.4, 8), (997, 21219.1, 2)], schema)

In [31]:
featurestore.create_featuregroup(sample_df, "enable_online_features_test")

computing descriptive statistics for : enable_online_features_test, version: 1
computing feature correlation for: enable_online_features_test, version: 1
computing feature histograms for: enable_online_features_test, version: 1
computing cluster analysis for: enable_online_features_test, version: 1
Registering feature metadata...
Registering feature metadata... [COMPLETE]
Writing feature data to offline feature group (Hive)...
Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Writing feature data to offline feature group (Hive)... [COMPLETE]
Feature group created successfully

#### Enable Online Feature Serving for Offline-Only feature group

In [32]:
featurestore.enable_featuregroup_online("enable_online_features_test")

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM enable_online_features_test_1 against offline feature store
Online Feature Serving enabled successfully for featuregroup: enable_online_features_test

#### DIsable Online Feature Serving for a feature group

In [33]:
featurestore.disable_featuregroup_online("enable_online_features_test")

Online Feature Serving disabled successfully for featuregroup: enable_online_features_test

### Insert into Offline/Online Feature Groups

When inserting data into a feature group you can control whether the data should be written only to the offline feature group, only to the online feature group, or to both, using the parameters `online=True` and `offline=True`:

#### Generate Sample Data

In [34]:
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, IntegerType, FloatType
sqlContext = SQLContext(sc)
schema = StructType([StructField("id", IntegerType(), True),
                     StructField("val_1", FloatType(), True),
                     StructField("val_2", IntegerType(), True)
                        ])
sample_df = sqlContext.createDataFrame([(111, 01251.52, 1), (958, 1919.4, 8), (697, 41219.1, 1)], schema)

#### Insert into Online Feature Group Only

In [35]:
featurestore.insert_into_featuregroup(sample_df, "online_featuregroup_test", 
                                      online=True, offline=False, mode="append")

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM online_featuregroup_test_1 against offline feature store
computing descriptive statistics for : online_featuregroup_test, version: 1
computing feature correlation for: online_featuregroup_test, version: 1
computing feature histograms for: online_featuregroup_test, version: 1
computing cluster analysis for: online_featuregroup_test, version: 1
Inserting data into online feature group online_featuregroup_test...
Inserting data into online feature group online_featuregroup_test... [COMPLETE]
Insertion into feature group was successful

#### Insert into Offline Feature Group Only

In [36]:
featurestore.insert_into_featuregroup(sample_df, "online_featuregroup_test", 
                                      online=False, offline=True, mode="append")

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM online_featuregroup_test_1 against offline feature store
computing descriptive statistics for : online_featuregroup_test, version: 1
computing feature correlation for: online_featuregroup_test, version: 1
computing feature histograms for: online_featuregroup_test, version: 1
computing cluster analysis for: online_featuregroup_test, version: 1
Inserting data into offline feature group online_featuregroup_test...
Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Inserting data into offline feature group online_featuregroup_test... [COMPLETE]
Insertion into feature group was successful

#### Insert into Online and Offline Feature Group

In [37]:
featurestore.insert_into_featuregroup(sample_df, "online_featuregroup_test", 
                                      online=True, offline=True, mode="overwrite")

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM online_featuregroup_test_1 against offline feature store
computing descriptive statistics for : online_featuregroup_test, version: 1
computing feature correlation for: online_featuregroup_test, version: 1
computing feature histograms for: online_featuregroup_test, version: 1
computing cluster analysis for: online_featuregroup_test, version: 1
Inserting data into offline feature group online_featuregroup_test...
Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Inserting data into offline feature group online_featuregroup_test... [COMPLETE]
Inserting data into online feature group online_featuregroup_test...
Inserting data into online feature group online_featuregroup_test... [COMPLETE]
Insertion into feature group was successful